In [1]:
from time import time

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import time
import torch
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import math
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST as FMNIST
from torchvision.datasets import EMNIST
import torchvision.transforms as transforms
import tensorflow as tf
import random as r
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook
import copy

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC

import matplotlib.pyplot as plt


In [2]:
path="../../Data/"
attacks_types = {
    'normal': 'normal','back': 'dos','buffer_overflow': 'u2r','ftp_write': 'r2l','guess_passwd': 'r2l',
'imap': 'r2l','ipsweep': 'probe','land': 'dos','loadmodule': 'u2r','multihop': 'r2l','neptune': 'dos',
'nmap': 'probe','perl': 'u2r','phf': 'r2l','pod': 'dos','portsweep': 'probe','rootkit': 'u2r','satan': 'probe',
'smurf': 'dos','spy': 'r2l','teardrop': 'dos','warezclient': 'r2l','warezmaster': 'r2l',
}
cols ="""duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,
urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,
num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,
count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,
diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,
dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,
dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate"""
  
columns =[]
for c in cols.split(','):
    if(c.strip()):
       columns.append(c.strip())
print(len(columns))
columns.append('target')
print(len(columns))

attack_categories=["dos","u2r","r2l",'probe','normal']
df = pd.read_csv(path+"kddcup.data_10_percent.gz", names = columns)
df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])
del df['target']
df.head()
num_cols = df._get_numeric_data().columns
  
cate_cols = list(set(df.columns)-set(num_cols))
cate_cols.remove('Attack Type')
def getuniquevalues(columnname):
    values={}
    i=0
    for entry in df[columnname]:
        if entry not in values:
            values[entry]=i
            i+=1
    return values
for col in cate_cols:
    df[col]=df[col].map(getuniquevalues(col))
data=df.to_numpy()
Y=df['Attack Type'].map(getuniquevalues('Attack Type'))
Y=Y.to_numpy()
X=data[:,:-1]
print(Y.shape)
print(X.shape)
print(getuniquevalues('Attack Type'))
def normalized(x,y):
    x, x_test, y, y_test = sklearn.model_selection.train_test_split(x, y, shuffle=True)
    scaler = sklearn.preprocessing.Normalizer().fit(x)
    x = scaler.transform(x)
    x_test = scaler.transform(x_test)

    # changes data to pytorch's tensors
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long()
    return x.numpy(), x_test.numpy(), y.numpy(), y_test.numpy(), scaler
xtrain, x_test, ytrain, y_test,scaler= normalized(X,Y)

41
42
(494021,)
(494021, 41)
{'normal': 0, 'u2r': 1, 'dos': 2, 'r2l': 3, 'probe': 4}


In [3]:
# def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config, epochs = None):
#     train_acc = []
#     test_acc = []
#     if epochs is not None:
#         param["epochs"] = epochs
#     #for i in tqdm_notebook(range(param["epochs"]), desc='epochs'):
#     for i in range(param["epochs"]):
#         train_acc.append(hdc.fit(traindata, trainlabels, param))
#         test_acc.append(hdc.test(testdata, testlabels))
#         # if len(train_acc) % 20 == 0:
#         #     print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
#         # if train_acc[-1] == 1:
#         #     print("Training converged!") 
#         #     print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
#         #     break

#     return np.asarray(train_acc), np.asarray(test_acc), i
def trainreploop(hdc,trainreps,trainencoded,trainlabels,param):
    # Do the train 
    for j in range(trainreps):
        train_acc = 100 * hdc.fit(trainencoded, trainlabels, param)
        # test_acc = 100 * hdc.test(testencoded, testlabels)
        # train_accs.append(train_acc)
        # test_accs.append(test_acc)
        print("Train: %.2f \t \t Test: "%(train_acc))
        if train_acc == 100:
            break
def regen(hdb,hdc,hde,amountDrop,traindata):
    var, orders = hdc.evaluateBasis()
    toDrop = orders[:amountDrop]
    # toMask = orders[-amountDrop:]
    # toDropVar = [var[i] for i in toDrop]
    print("Variances stats: max %.2f, min %.2f, mean %.2f"%(max(var),min(var),np.mean(var)))
    #print("Dropping first %f percent of ineffective basis, with stats: max %f, min %f, mean %f"\
    #      %(percentDrop, max(toDropVar),min(toDropVar),np.mean(toDropVar)))
    hdb.updateBasis(toDrop)
    hde.updateBasis(hdb.basis)
    hdc.updateClasses()
    return hde.encodeData(traindata)

In [4]:
def train_neural_ed(traindata, trainlabels, testdata, testlabels,
                   D, # initial baseline
                   regenloops,  # list of effective dimensions to reach 
                   percentDrop, # drop/regen rate 
                   trainreps, # # iterations per regen  
                   param):

    param["D"] = D
    
    # Initialize basis & classifier
    hdb = HDB.HD_basis(HDB.Generator.Vanilla, param)
    basis = hdb.getBasis()
    param = hdb.getParam()
    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traindata)
    # testencoded = hde.encodeData(testdata)
    # Initialize classifier
    # train_accs = []
    # test_accs = []
    hdc = HDC.HD_classifier(param["D"], param["nClasses"], 0)

    # Prepare setting for train
    amountDrop = int(percentDrop * hdc.D)
    print("Updating times:", regenloops)

    # early_stopping_steps = 1000 # earlystopping is "turned off"

    # #es_count = 0
    # max_test = 0
    # best = None
    # best_idx = 0
    
    # # Checkpoints
    # checkpoints = []

    for i in range(regenloops+1): # For each eDs to reach, will checkpoints
        print("regenloop: " + str(i))
        # Do the train 
        trainreploop(hdc,trainreps,trainencoded,trainlabels,param)
        # for j in range(iter_per_update):
        #     train_acc = 100 * hdc.fit(trainencoded, trainlabels, param)
        #     # test_acc = 100 * hdc.test(testencoded, testlabels)
        #     # train_accs.append(train_acc)
        #     # test_accs.append(test_acc)
        #     print("Train: %.2f \t \t Test: "%(train_acc))
        #     if train_acc == 100:
        #         break
         
        # if train_acc == 100:
        #     print("Train converged! taking snippit in checkpoints")
        #     hdb_ck = copy.deepcopy(hdb)
        #     hdc_ck = copy.deepcopy(hdc)
        #     _, post_test_accs, _ = train(hdc_ck, trainencoded, trainlabels, testencoded, testlabels, param, epochs = 50)
            # checkpoints.append((i+1, (D + (i)*amountDrop), 
            #                     hdb_ck, hdc_ck, 
            #                     max(test_accs[-iter_per_update:]), max(post_test_accs)))

        # if test_accs[-1] >= max_test:
        #     es_count = 0
        #     best = copy.deepcopy(hdc)
        #     best_idx = len(test_accs)
        # else:
        #     es_count += 1
        # if es_count > early_stopping_steps:
        #     print("Early stopping initiated, best stores the best hdc currently")
        #     break
        
        # if i in regenTimes:
        #     print("Checkpoint made!")
        #     hdb_ck = copy.deepcopy(hdb)
        #     hdc_ck = copy.deepcopy(hdc)
        #     _, post_test_accs, _ = train(hdc_ck, trainencoded, trainlabels, testencoded, testlabels, param, epochs = 50)
        #     checkpoints.append((D, (D + (i)*amountDrop), 
        #                         None, None, #hdb_ck, hdc_ck, 
        #                         max(test_accs[-iter_per_update:]), max(post_test_accs)))
        if i==regenloops:
            return hdc,hde
        print("regeneration")
        # Do the regeneration
        # var, orders = hdc.evaluateBasis()
        # toDrop = orders[:amountDrop]
        # toMask = orders[-amountDrop:]
        # toDropVar = [var[i] for i in toDrop]
        # print("Variances stats: max %.2f, min %.2f, mean %.2f"%(max(var),min(var),np.mean(var)))
        # #print("Dropping first %f percent of ineffective basis, with stats: max %f, min %f, mean %f"\
        # #      %(percentDrop, max(toDropVar),min(toDropVar),np.mean(toDropVar)))
        # hdb.updateBasis(toDrop)
        # hde.updateBasis(hdb.basis)
        # trainencoded = hde.encodeData(traindata)
        # testencoded = hde.encodeData(testdata)
        # hdc.updateClasses()
        trainencoded=regen(hdb,hdc,hde,amountDrop,traindata)
    return "error","error"

In [6]:
D=100
percentDrop=0.1
regenloops=5
trainreps = 2
param = Config.config
param["nFeatures"] = xtrain.shape[1]
param["nClasses"] = len(df['Attack Type'].unique())

#Ds = [200, 500, 750, 1000, 1500, 2000, 2500, 3000, 4000]
#percentDrops = [0.05]
#iter_per_updates = [1, 2, 3, 4]


hdc=None
# for iter_per_update in iter_per_updates:
print("Current config:", D, percentDrop, trainreps)
hdc,hde = train_neural_ed(xtrain, ytrain, x_test, y_test,
        D, # initial baseline
        regenloops,
        percentDrop, # drop/regen rate 
        trainreps, # # iterations per regen  
        param)

Current config: 100 0.1 2
Updating times: 5
regenloop: 0
Train: 98.34 	 	 Test: 
Train: 98.60 	 	 Test: 
regeneration
Variances stats: max 0.05, min 0.00, mean 0.01
Updating basis......
regenloop: 1
Train: 98.54 	 	 Test: 
Train: 98.70 	 	 Test: 
regeneration
Variances stats: max 0.03, min 0.00, mean 0.01
Updating basis......
regenloop: 2
Train: 98.58 	 	 Test: 
Train: 98.70 	 	 Test: 
regeneration
Variances stats: max 0.03, min 0.00, mean 0.01
Updating basis......
regenloop: 3
Train: 98.59 	 	 Test: 
Train: 98.74 	 	 Test: 
regeneration
Variances stats: max 0.04, min 0.00, mean 0.01
Updating basis......
regenloop: 4
Train: 98.61 	 	 Test: 
Train: 98.73 	 	 Test: 
regeneration
Variances stats: max 0.04, min 0.00, mean 0.01
Updating basis......
regenloop: 5
Train: 98.59 	 	 Test: 
Train: 98.75 	 	 Test: 


In [28]:
def predict(hdc,hde,x):
    trainencoded=hde.encodeData(x)
    return hdc.predict(trainencoded)

In [23]:
print(np.unique(Y))

[0 1 2 3 4]


In [29]:
for i in range (0,len(np.unique(Y))):
    yhat=predict(hdc,hde,x_test[y_test==i])
    acc = (yhat==i).mean()
    print(yhat==i)

AttributeError: 'bool' object has no attribute 'mean'

In [32]:
for i in range (0,len(np.unique(Y))):
    yhat=np.array(predict(hdc,hde,x_test[y_test==i]))
    acc = (yhat==i).mean()
    print(acc)

0.9780424264979531
0.0
0.9905367922989068
0.8692307692307693
0.9139676113360324


In [33]:
param

{'data_location': './dataset/',
 'directory': 'smart_home',
 'dataset': 'smart_home_split',
 'D': 100,
 'vector': 'Gaussian',
 'mu': 0,
 'sigma': 1,
 'binarize': 0,
 'lr': 0.037,
 'sparse': 0,
 's': 0.1,
 'binaryModel': 0,
 'checkpoints': False,
 'width': None,
 'height': None,
 'nLayers': 5,
 'uniform_dim': 1,
 'uniform_ker': 1,
 'dArr': None,
 'k': 3,
 'kArr': None,
 'one_shot': 0,
 'data_percentages': [1.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5],
 'train_percent': 1,
 'dropout': 0,
 'drop_percentages': [0, 0.1, 0.2, 0.5],
 'dropout_rate': 0,
 'update_type': <Update_T.FULL: 1>,
 'masked': False,
 'iter_per_trial': 3,
 'iter_per_encoding': 5,
 'epochs': 250,
 'nFeatures': 41,
 'nClasses': 5,
 'id': '5420',
 'gen_type': <Generator.Vanilla: 1>}

In [12]:

    # yhat=np.array([row.numpy().argmax() for row in yhat])
    # acc = (yhat==i).mean()
    # print('class '+str(i)+' accuracy: ' +str(acc))
    # print('points: '+ str(len(yhat)))

(24616, 41)


AssertionError: 

In [51]:
class NeuralHD:
    def __init__(self, classes : int, features : int, dim : int = 400):
        self.param=Config.config
        self.param['nClasses'] = classes
        self.param['nFeatures']= features
        self.param['D']=dim
        self.hde=None
        self.hdc=None
    def __call__(self, x : torch.Tensor):
        assert hde!=None and hdc!=None
        return self.predict(x)
    def predict(self,x):
        trainencoded=self.hde.encodeData(x)
        return np.array(self.hdc.predict(trainencoded))
    def fit(self,traindata, trainlabels,
                   trainreps,
                   regenloops,  # list of effective dimensions to reach 
                   percentDrop # drop/regen rate 
                    ):
        
        # Initialize basis & classifier
        hdb = HDB.HD_basis(HDB.Generator.Vanilla, self.param)
        basis = hdb.getBasis()
        self.param = hdb.getParam()
        self.hde = HDE.HD_encoder(basis)
        trainencoded = self.hde.encodeData(traindata)
        self.hdc = HDC.HD_classifier(self.param["D"], self.param["nClasses"], 0)

        # Prepare setting for train
        amountDrop = int(percentDrop * self.hdc.D)#self.param.D?
        print("Updating times:", regenloops)

        for i in range(regenloops+1): # For each eDs to reach, will checkpoints
            print("regenloop: " + str(i))
            # Do the train 
            self.trainreploop(trainreps,trainencoded,trainlabels)
            if i==regenloops:
                return self.hdc,self.hde
            print("regeneration")
            trainencoded=self.regen(hdb,amountDrop,traindata)
        return "error","error"
    
    def trainreploop(self,trainreps,trainencoded,trainlabels):
        # Do the train 
        for j in range(trainreps):
            train_acc = 100 * self.hdc.fit(trainencoded, trainlabels, self.param)
            # test_acc = 100 * hdc.test(testencoded, testlabels)
            # train_accs.append(train_acc)
            # test_accs.append(test_acc)
            print("Train: %.2f \t \t Test: "%(train_acc))
            if train_acc == 100:
                break
    def regen(self,hdb,amountDrop,traindata):
        var, orders = self.hdc.evaluateBasis()
        toDrop = orders[:amountDrop]
        # toMask = orders[-amountDrop:]
        # toDropVar = [var[i] for i in toDrop]
        print("Variances stats: max %.2f, min %.2f, mean %.2f"%(max(var),min(var),np.mean(var)))
        #print("Dropping first %f percent of ineffective basis, with stats: max %f, min %f, mean %f"\
        #      %(percentDrop, max(toDropVar),min(toDropVar),np.mean(toDropVar)))
        hdb.updateBasis(toDrop)
        self.hde.updateBasis(hdb.basis)
        self.hdc.updateClasses()
        return self.hde.encodeData(traindata)    

In [20]:
hdc.counts

array([1., 1., 1., 1., 1.])

In [53]:
model=NeuralHD(5,xtrain.shape[1],300)

In [54]:
model.fit(xtrain,ytrain,2,5,.1)

Updating times: 5
regenloop: 0
Train: 98.34 	 	 Test: 
Train: 98.66 	 	 Test: 
regeneration
Variances stats: max 0.03, min 0.00, mean 0.00
Updating basis......
regenloop: 1
Train: 98.56 	 	 Test: 
Train: 98.71 	 	 Test: 
regeneration
Variances stats: max 0.02, min 0.00, mean 0.00
Updating basis......
regenloop: 2
Train: 98.54 	 	 Test: 
Train: 98.73 	 	 Test: 
regeneration
Variances stats: max 0.02, min 0.00, mean 0.00
Updating basis......
regenloop: 3
Train: 98.59 	 	 Test: 
Train: 98.75 	 	 Test: 
regeneration
Variances stats: max 0.02, min 0.00, mean 0.00
Updating basis......
regenloop: 4
Train: 98.58 	 	 Test: 
Train: 98.71 	 	 Test: 
regeneration
Variances stats: max 0.02, min 0.00, mean 0.00
Updating basis......
regenloop: 5
Train: 98.58 	 	 Test: 
Train: 98.75 	 	 Test: 


(<HD_classifier.HD_classifier at 0x108f4bfd0>,
 <HD_encoder.HD_encoder at 0x108d12e50>)

In [56]:
for i in range (0,len(df['Attack Type'].unique())):
    yhat= model(x_test[y_test==i])
    acc = (yhat==i).mean()
    print('class '+str(i)+' accuracy: ' +str(acc))
    print('points: '+ str(len(yhat)))

class 0 accuracy: 0.9983045941363768
points: 24183
class 1 accuracy: 0.0
points: 11
class 2 accuracy: 0.9923009463207701
points: 98064
class 3 accuracy: 0.15384615384615385
points: 260
class 4 accuracy: 0.8026315789473685
points: 988


In [46]:
type(model.hdc)

NoneType

41